In [ ]:
!pip install transformers peft accelerate datasets torch bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes


SyntaxError: invalid syntax (<ipython-input-9-cb66a89e4e3d>, line 2)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb

# Model name
model_name = "deepseek-ai/deepseek-math-7b-instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Load model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,   # Use bf16 for better precision
    load_in_4bit=True,            # Enables 4-bit quantization
    device_map="auto",
    trust_remote_code=True
)

print("Model loaded successfully with 4-bit quantization!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully with 4-bit quantization!


In [ ]:
from datasets import Dataset

# Extended Math Meme Correction Dataset
meme_data = [
    {"input": "8 ÷ 2(2+2) = 1?", "output": "8 ÷ 2(2+2) = 16 (PEMDAS: 8 ÷ 2 × 4 = 16)"},
    {"input": "52 = 10", "output": "5² = 25 (Exponentiation: 5 squared is 5 × 5, not 5 × 2)"},
    {"input": "100 ÷ 5 + 5 = 1?", "output": "100 ÷ 5 + 5 = 25 (100 ÷ 5 = 20, then 20 + 5 = 25)"},
    {"input": "2 + 2 × 2 = 8?", "output": "2 + 2 × 2 = 6 (PEMDAS: 2 × 2 = 4, then 2 + 4 = 6)"},
    {"input": "6 ÷ 2(1+2) = 1?", "output": "6 ÷ 2(1+2) = 9 (Follow PEMDAS: 6 ÷ 2 × 3 = 9)"},
    {"input": "0.999... ≠ 1?", "output": "0.999... = 1 (1/3 = 0.333..., so 3 × 1/3 = 1)"},
    {"input": "√(-1) is real?", "output": "√(-1) is imaginary (It equals i, not a real number)"},
    {"input": "100 - 50 ÷ 5 = 10?", "output": "100 - 50 ÷ 5 = 90 (50 ÷ 5 = 10, then 100 - 10 = 90)"},
    {"input": "1 + 1 = 3?", "output": "1 + 1 = 2 (Basic arithmetic)"},
    {"input": "10 ÷ (5 - 5) = 2?", "output": "Undefined (Division by zero is not possible)"},
    {"input": "3 + 3 × 3 = 18?", "output": "3 + 3 × 3 = 12 (Multiplication first: 3 × 3 = 9, then 9 + 3 = 12)"},
    {"input": "25 × 0 = 25?", "output": "25 × 0 = 0 (Any number multiplied by zero is zero)"},
    {"input": "4 ÷ 1/2 = 2?", "output": "4 ÷ 1/2 = 8 (Dividing by 1/2 is the same as multiplying by 2)"},
    {"input": "5 + 5 × 0 = 5?", "output": "5 + 5 × 0 = 5 (Multiplication first: 5 × 0 = 0, then 5 + 0 = 5)"},
    {"input": "2³ = 6?", "output": "2³ = 8 (Exponentiation: 2 × 2 × 2 = 8)"},
    {"input": "7 + 3 = 9?", "output": "7 + 3 = 10 (Basic addition)"},
    {"input": "1000 ÷ 10 = 1?", "output": "1000 ÷ 10 = 100 (Basic division)"},
    {"input": "0.1 + 0.2 = 0.4?", "output": "0.1 + 0.2 = 0.3 (Floating-point precision issues aside, 0.1 + 0.2 = 0.3)"},
    {"input": "2(3+5) = 16?", "output": "2(3+5) = 16 (Correct: 2 × (3+5) = 16)"},
    {"input": "5 × 2 = 20?", "output": "5 × 2 = 10 (Multiplication error)"},
    {"input": "(4+6) × 2 = 14?", "output": "(4+6) × 2 = 20 (Addition first, then multiplication)"},
    {"input": "30 ÷ 5 × 2 = 3?", "output": "30 ÷ 5 × 2 = 12 (Follow left-to-right: 30 ÷ 5 = 6, then 6 × 2 = 12)"},
    {"input": "9² = 72?", "output": "9² = 81 (Exponentiation: 9 × 9 = 81)"},
    {"input": "4 × 3 + 2 = 14?", "output": "4 × 3 + 2 = 14 (Multiplication first: 4 × 3 = 12, then 12 + 2 = 14)"},
    {"input": "8 × 7 = 54?", "output": "8 × 7 = 56 (Multiplication mistake)"},
    {"input": "(6+2)² = 36?", "output": "(6+2)² = 64 (Addition first: 6+2 = 8, then 8² = 64)"},
    {"input": "200 ÷ 4 × 2 = 25?", "output": "200 ÷ 4 × 2 = 100 (200 ÷ 4 = 50, then 50 × 2 = 100)"},
    {"input": "10 - 3 × 2 = 14?", "output": "10 - 3 × 2 = 4 (Multiplication first: 3 × 2 = 6, then 10 - 6 = 4)"},
    {"input": "(3+4) × 2 = 10?", "output": "(3+4) × 2 = 14 (Addition first, then multiply)"},
]

# Convert to Hugging Face dataset format
dataset = Dataset.from_list(meme_data)

print("Dataset created successfully!")

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding token

def tokenize_function(sample):
    """Tokenizes input-output pairs"""
    input_text = f"Wrong: {sample['input']} Correct:"
    output_text = sample["output"]

    # Tokenize input and output
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    outputs = tokenizer(output_text, truncation=True, padding="max_length", max_length=128)

    # Replace padding tokens in labels with -100 to ignore during loss calculation
    labels = outputs["input_ids"]
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]

    # Return tokenized format
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels  # Masked labels
    }

# Tokenize entire dataset
tokenized_dataset = dataset.map(tokenize_function)

print("Dataset tokenized successfully!")


Dataset created successfully!


Map:   0%|          | 0/29 [00:00<?, ? examples/s]

Dataset tokenized successfully!


In [ ]:
# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as padding token

def tokenize_function(sample):
    """Tokenizes input-output pairs"""
    input_text = f"Wrong: {sample['input']} Correct:"
    output_text = sample["output"]

    # Tokenize input and output
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    outputs = tokenizer(output_text, truncation=True, padding="max_length", max_length=128)

    # Replace padding tokens in labels with -100 to ignore during loss calculation
    labels = outputs["input_ids"]
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]

    # Return tokenized format
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels  # Masked labels
    }

# Tokenize entire dataset
tokenized_dataset = dataset.map(tokenize_function)

print("Dataset tokenized successfully!")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset tokenized successfully!


In [ ]:
from peft import prepare_model_for_kbit_training

# LoRA Configuration
lora_config = LoraConfig(
    r=16,                   # Rank of update matrices
    lora_alpha=32,          # Alpha scaling factor
    lora_dropout=0.05,      # Dropout for regularization
    bias="none",
    target_modules=["q_proj", "v_proj"],  # Target Q and V projection layers
    task_type="CAUSAL_LM",  # Causal Language Modeling
)

# Prepare model for LoRA training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

print("LoRA fine-tuning applied successfully!")


LoRA fine-tuning applied successfully!


In [ ]:
training_args = TrainingArguments(
    output_dir="./deepseek_math_finetuned",
     report_to="none",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,         # Train for 3 epochs
    learning_rate=5e-5,         # Learning rate
    fp16=True,                  # Enable mixed precision
    logging_steps=10,           # Log every 10 steps
    save_steps=500,             # Save checkpoint every 500 steps
    save_total_limit=2,         # Keep only 2 checkpoints
    evaluation_strategy="no",
    logging_dir="./logs",
    push_to_hub=False,          # Set to True if pushing to Hugging Face
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer, DataCollatorForSeq2Seq

# Data collator for padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train model
trainer.train()


<ipython-input-16-2c63d23baa72>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.war

Step,Training Loss
10,13.468300
20,11.180200
30,9.454500
40,9.684900
50,8.186300
60,7.343900
70,6.609700
80,7.602400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=87, training_loss=8.98895465368512, metrics={'train_runtime': 130.5973, 'train_samples_per_second': 0.666, 'train_steps_per_second': 0.666, 'total_flos': 434223795142656.0, 'train_loss': 8.98895465368512, 'epoch': 3.0})

In [ ]:
trainer.save_model("./deepseek_math_finetuned")
tokenizer.save_pretrained("./deepseek_math_finetuned")

print("Fine-tuned model saved successfully!")


Fine-tuned model saved successfully!


In [ ]:
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU/CPU
model.to(device)

def test_model(question):
    """Generates a corrected answer using the fine-tuned model."""
    input_text = f"Wrong: {question} Correct:"

    # Tokenize
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate response
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=100,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode response
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract corrected answer
    corrected_answer = response.split("Correct:")[-1].strip()

    return corrected_answer

# Test with a math meme
test_question = "8 ÷ 2(2+2) = 1?"
print(f"❌ Incorrect: {test_question}")
print(f"✅ Corrected: {test_model(test_question)}")


❌ Incorrect: 8 ÷ 2(2+2) = 1?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more detai

✅ Corrected: 8 ÷ 2(2+2) = 8 ÷ 2(4) = 8 ÷ 8 = 1
